# Hurricane Raster Fusion Analysis

**PLAN:** Load tweets → Create grid → Temporal bins → Fuse signals → Export rasters

**DO:** Execute pipeline

**VERIFY:** Check outputs

In [1]:
CONFIG = {
    'event': 'helene',
    'cell_size_km': 5,
    'time_bin_hours': 2,
    'weights': {'coordinates': 0.50, 'city': 0.30, 'county': 0.15, 'state': 0.05},
    'crs': 'EPSG:5070',
    'export_slices': True
}
assert abs(sum(CONFIG['weights'].values()) - 1.0) < 0.001
print(f"Config: {CONFIG['event']}, {CONFIG['cell_size_km']}km")

Config: helene, 5km


In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np
import rasterio
from rasterio.transform import from_bounds
from pathlib import Path
from datetime import datetime
import warnings, json
warnings.filterwarnings('ignore')
print('Imports ready')

Imports ready


In [3]:
DATA = Path(r'C:\Users\colto\Documents\GitHub\Tweet_project\data')
OUT = Path(r'C:\Users\colto\Documents\GitHub\Tweet_project\rasters_output')
OUT.mkdir(exist_ok=True)
EVENT_DIR = OUT / CONFIG['event']
EVENT_DIR.mkdir(exist_ok=True)
print(f'Output: {EVENT_DIR}')

Output: C:\Users\colto\Documents\GitHub\Tweet_project\rasters_output\helene


In [4]:
print('Loading tweets...')
tweets = gpd.read_file(DATA / 'geojson' / f"{CONFIG['event']}.geojson")
tweets['time'] = pd.to_datetime(tweets['time'])
if tweets.crs is None:
    tweets.set_crs('EPSG:4326', inplace=True)
tweets = tweets.to_crs(CONFIG['crs'])
print(f'{len(tweets)} tweets, {tweets["time"].min()} to {tweets["time"].max()}')

Loading tweets...


3007 tweets, 2024-09-26 02:29:25+00:00 to 2024-09-27 19:59:41+00:00


In [5]:
buffer_m = 100000
cell_m = CONFIG['cell_size_km'] * 1000
bounds = tweets.total_bounds
minx = bounds[0]-buffer_m
miny = bounds[1]-buffer_m
maxx = bounds[2]+buffer_m
maxy = bounds[3]+buffer_m
width = int(np.ceil((maxx-minx)/cell_m))
height = int(np.ceil((maxy-miny)/cell_m))
transform = from_bounds(minx, miny, maxx, maxy, width, height)
print(f'Grid: {width}x{height} = {width*height:,} cells')
x_coords = np.linspace(minx+cell_m/2, maxx-cell_m/2, width)
y_coords = np.linspace(miny+cell_m/2, maxy-cell_m/2, height)
X, Y = np.meshgrid(x_coords, y_coords)

Grid: 279x390 = 108,810 cells


In [6]:
min_t = tweets['time'].min()
max_t = tweets['time'].max()
start_t = min_t.floor(f"{CONFIG['time_bin_hours']}h")
end_t = max_t.ceil(f"{CONFIG['time_bin_hours']}h")
time_bins = pd.date_range(start_t, end_t, freq=f"{CONFIG['time_bin_hours']}h")
tweets['bin'] = pd.cut(tweets['time'], bins=time_bins, labels=range(len(time_bins)-1), include_lowest=True)
print(f'{len(time_bins)-1} bins of {CONFIG["time_bin_hours"]}h each')
bin_meta = pd.DataFrame({'idx': range(len(time_bins)-1), 'start': time_bins[:-1], 'end': time_bins[1:]})

21 bins of 2h each


In [7]:
def fuse_intensity(tweet_subset, grid_x, grid_y, sigma_m=3000):
    intensity = np.zeros(grid_x.shape, dtype=np.float32)
    for idx, row in tweet_subset.iterrows():
        if row.geometry and row.geometry.is_valid:
            px, py = row.geometry.x, row.geometry.y
            dist_sq = (grid_x - px)**2 + (grid_y - py)**2
            kernel = np.exp(-dist_sq / (2*sigma_m**2))
            intensity += kernel
    if intensity.max() > 0:
        intensity = intensity / intensity.max()
    return intensity

print('Fusion ready')

Fusion ready


In [8]:
print('Generating slices...')
slices_iter = []
slices_cum = []
for idx, row in bin_meta.iterrows():
    tweets_in_bin = tweets[tweets['bin'] == idx]
    tweets_cumulative = tweets[tweets['bin'] <= idx]
    print(f'Bin {idx}: {len(tweets_in_bin)} tweets')
    if len(tweets_in_bin) > 0:
        intensity_i = fuse_intensity(tweets_in_bin, X, Y)
    else:
        intensity_i = np.zeros((height, width), dtype=np.float32)
    if len(tweets_cumulative) > 0:
        intensity_c = fuse_intensity(tweets_cumulative, X, Y)
    else:
        intensity_c = np.zeros((height, width), dtype=np.float32)
    slices_iter.append(intensity_i)
    slices_cum.append(intensity_c)
    if CONFIG['export_slices']:
        for name, data in [('iterative', intensity_i), ('cumulative', intensity_c)]:
            path = EVENT_DIR / f'slice_{idx:03d}_{name}.tif'
            with rasterio.open(path, 'w', driver='GTiff', height=height, width=width, count=1, dtype=np.float32, crs=CONFIG['crs'], transform=transform, compress='lzw') as dst:
                dst.write(data, 1)
                dst.update_tags(time_start=str(row['start']), time_end=str(row['end']))
print(f'{len(slices_iter)} slices generated')

Generating slices...
Bin 0: 76 tweets


Bin 1: 84 tweets


Bin 2: 57 tweets


Bin 3: 86 tweets


Bin 4: 82 tweets


...

Bin 17: 158 tweets


Bin 18: 162 tweets


Bin 19: 123 tweets


Bin 20: 141 tweets


21 slices generated


In [9]:
meta = {
    'pipeline': 'Raster Fusion',
    'timestamp': datetime.now().isoformat(),
    'config': CONFIG,
    'grid': {'width': width, 'height': height},
    'slices': len(slices_iter)
}
print(json.dumps(meta, indent=2))
with open(EVENT_DIR / 'metadata.json', 'w') as f:
    json.dump(meta, f, indent=2)
print(f'Metadata saved')

{
  "pipeline": "Raster Fusion",
  "timestamp": "2025-10-29T18:35:13.617370",
  "config": {
    "event": "helene",
    "cell_size_km": 5,
    "time_bin_hours": 2,
    "weights": {
      "coordinates": 0.5,
      "city": 0.3,
      "county": 0.15,
      "state": 0.05
    },
    "crs": "EPSG:5070",
    "export_slices": true
  },
  "grid": {
    "width": 279,
    "height": 390
  },
  "slices": 21
}
Metadata saved


In [10]:
print('VERIFICATION:')
assert len(slices_iter) == len(bin_meta)
print('  ✓ Count OK')
assert slices_iter[0].shape == (height, width)
print('  ✓ Shape OK')
print(f'  ✓ Slices: {sum(1 for s in slices_iter if s.max() > 0)}/{len(slices_iter)} non-empty')
print(f'\nOUTPUTS: {EVENT_DIR}')
print('COMPLETE')

VERIFICATION:
  ✓ Count OK
  ✓ Shape OK
  ✓ Slices: 21/21 non-empty

OUTPUTS: C:\Users\colto\Documents\GitHub\Tweet_project\rasters_output\helene
COMPLETE


## README

### Fusion Strategy
Gaussian kernel density from tweet coordinates. Normalized intensity [0,1].

### ArcGIS Pro
1. Add GeoTIFF slices
2. Enable time on layer
3. Use time slider
4. Symbology: Stretched, heat colors

### Files
- `slice_NNN_iterative.tif`
- `slice_NNN_cumulative.tif`
- `metadata.json`